In [3]:
import pandas as pd
import spacy
import random 
import json
import os
from pathlib import Path
from spacy.training import Example
from spacy.tokens import DocBin
from spacy.tokens import Doc
from spacy.language import Language
from spacy.tokens import Span
from spacy.pipeline import EntityRuler
from spacy.matcher import PhraseMatcher


os.chdir(r'/home/dlin1765/repos/tekken-move-guesser')
with open('annotations.json', 'r') as raw:
    data = json.load(raw)


In [4]:
# create new training data 



In [5]:
# python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy 

In [6]:
from types import NoneType


entity_names = ['MOVE_TYPE', 'MOVE_HIT_PROPERTY', 'MOVE_BLOCK_PROPERTY', 'MOVE_PROPERTY', 'RANGE', 'LIMB', 'MOVE_STATE', 'SPECIFIC_INPUT', 'MOVE_NAME', 'STRING_DETAIL']
train_data = data['annotations']
#train_data = [tuple(i) for i in train_data]
new_list = []

# remove all NoneType data that may have been introduced
for x in train_data[:]:
    if isinstance(x, NoneType):
        train_data.remove(x)

# format the data into the required format
for i in range(0, len(train_data)):
    new_list = []
    tupled_data = tuple(train_data[i])
    for lists in train_data[i][1].get('entities'):
        new_list.append(tuple(lists))
    new_dict = {
        'entities': new_list
    }
    tupled_data = (train_data[i][0], new_dict)
    train_data[i] = tupled_data

print(train_data)
    

[('Low ', {'entities': [(0, 3, 'MOVE_HIT_PROPERTY')]}), ('Mid', {'entities': [(0, 3, 'MOVE_HIT_PROPERTY')]}), ('High', {'entities': [(0, 4, 'MOVE_HIT_PROPERTY')]}), ('Guard crush/unblockable', {'entities': [(0, 23, 'MOVE_HIT_PROPERTY')]}), ('Counterhit', {'entities': [(0, 10, 'MOVE_HIT_PROPERTY')]}), ('Tornado', {'entities': [(0, 7, 'MOVE_HIT_PROPERTY')]}), ('launch', {'entities': [(0, 6, 'MOVE_HIT_PROPERTY')]}), ('Mini combo ', {'entities': [(0, 10, 'MOVE_HIT_PROPERTY')]}), ('Flip over ', {'entities': [(0, 9, 'MOVE_HIT_PROPERTY')]}), ('Heat engager', {'entities': [(0, 12, 'MOVE_TYPE')]}), ('Parry', {'entities': [(0, 5, 'MOVE_TYPE')]}), ('String', {'entities': [(0, 6, 'MOVE_TYPE')]}), ('Power crush', {'entities': [(0, 11, 'MOVE_TYPE')]}), ('Stance ', {'entities': [(0, 6, 'MOVE_TYPE')]}), ('plus ', {'entities': [(0, 4, 'MOVE_BLOCK_PROPERTY')]}), ('minus ', {'entities': [(0, 5, 'MOVE_BLOCK_PROPERTY')]}), ('evasive', {'entities': [(0, 7, 'MOVE_PROPERTY')]}), ('transition', {'entities': [(

In [7]:
nlp = spacy.load("en_core_web_sm") # load other spacy model
db = DocBin() 

#format data into spacy format
for text, annot in train_data: 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]: 
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc) 

os.chdir(r'/home/dlin1765/repos/tekken-move-guesser')
db.to_disk("./train.spacy") 

In [8]:
# add entity ruler pipeline to the nlp
# ruler = EntityRuler(nlp, overwrite_ents=True)
# ruler = nlp.add_pipe("entity_ruler")
import re
# config = {
#    "phrase_matcher_attr": None,
#    "validate": True,
#    "overwrite_ents": True,
#    "ent_id_sep": "||",
# }
# nlpTest = spacy.load("en_core_web_sm")

#ruler = nlpTest.add_pipe("entity_ruler", config=config)

# ^(?:(?:[udbnf]{0,10}|([udbnf]*[\s,.+]*){0,10}|(sway)?|(dpd)?)?[\s,]*[1-4](?:\s*[,+-]\s*[1-4])*)+$|^((?:[1-4](?:\s*[,+-]\s*[1-4])*)+)$



# specificMoveNames = ['lethal sky rocket', 'jfsr', 'skyrocket', 'sky rocket', '10 hit', '10hit', 'ten hit']
# directionInputs = ['n', 'NOINPUT', 'neutral', 'f', 'forward', 'df', 'down forward', 'downforward', 'down-forward', 'd', 'down', 'db', 'downback', 'down back', 'down-back', 'b', 'back', 'ub', 'upback', 'up back', 'up-back', 'u', 'up', 'uf', 'up forward', 'upforward', 'up-forward', 'ss', 'sidestep', 'side step', 'qcf' 'quarter circle forward', 'quartercircleforward', 'quarter-circle-forward', 'qcb', 'quarter circle back', 'quarter-circle-back', 'quartercircleback', 'ff', 'forwardforward', 'f,f', 'f, f', 'fff', 'f,f,f', 'f, f, f', 'while running', 'wr','triple forward', 'forwardforwardforward', 'bf', 'back forward', 'backforward', 'bnf', 'b,n,f', 'b, n, f', 'back neutral forward', 'backneutralforward', 'back,neutral,forward', 'ws', 'while standing', 'while rising', 'w.s', 'w,s', 'w s', 'whilestanding', 'whilerising', 'fc', 'full crouch', 'fullcrouch', 'cd', 'crouch dash', 'crouchdash', 'bt', 'backturn', 'back turn', 'back-turn']
# delimiters = ['',' ', '+', ' + ', ' +', '+ ', '.', ' . ', ' .', '. ', ',', ' , ', ', ', ' ,', '-', ' - ', '- ', ' -',]
# allPossibleInputs = ['1', '2', '3', '4']

# patterns1 = [
#             {"TEXT": {"REGEX": "(?:(?:[udbnf]{0,10}|([udbnf]+[\s,.+]*){0,10}|(sway)?|(dpd)?)?[\s,]*[1-4](?:\s*[,+-]\s*[1-4])*)+|^((?:[1-4](?:\s*[,+-]\s*[1-4])*)+)"}}
#             ]

# patterns = [{"label": "SPECIFIC_MOVE", "pattern": [
#             {"TEXT": {"REGEX": "(?:(?:[udbnf]{0,10}|([udbnf]+[\s,.+]*){0,10}|(sway)?|(dpd)?)?[\s,]*[1-4](?:\s*[,+-]\s*[1-4])*)+|((?:[1-4](?:\s*[,+-]\s*[1-4])*)+)"}}
#             ]}]




#ruler.add_patterns([{"TEXT": {"REGEX": "(?:(?:[udbnf]{0,10}|([udbnf]+[\s,.+]*){0,10}|(sway)?|(dpd)?)?[\s,]*[1-4](?:\s*[,+-]\s*[1-4])*)+|^((?:[1-4](?:\s*[,+-]\s*[1-4])*)+)"}}])
#ruler.add_patterns(patterns)
#testDoc = nlpTest("this is a sentence about drag b 3,2, or written as b3,2")

@Language.component("specificMoves")
def specificMoves(doc):
    new_ents = []
    expression = "(?:(?:[udbnf]{0,10}|([udbnf]+[\s,.+]*){0,10}|(sway)?|(dpd)?)?[\s,]*[1-4](?:\s*[,+-]\s*[1-4])*)+|((?:[1-4](?:\s*[,+-]\s*[1-4])*)+)"
    for match in re.finditer(expression, doc.text):
        start, end = match.span()
        span = doc.char_span(start, end, label = "SPECIFIC_MOVE")
        if span is not None:
            new_ents.append(span)

    doc.ents = list(doc.ents) + new_ents
    return doc



In [9]:
import spacy.displacy
#adding a tagger and a parser to  

#nlp1 = spacy.load(r"/home/dlin1765/repos/tekken-move-guesser/output/model-best") 


nlp1 = spacy.load(r"/home/dlin1765/repos/tekken-move-guesser/output/model-last") 
# new model correctly finds all instances of specific moves
doc8 = nlp1("claudio b2,3,4")
doc9 = nlp1("claudio df2,3,4")
doc10 = nlp1("claudio df 2+3 with outer stuff too")

spacy.displacy.render(doc8, style="ent", jupyter=True)
spacy.displacy.render(doc9, style="ent", jupyter=True)
spacy.displacy.render(doc10, style="ent", jupyter=True)

# kind of messes up certain properties, like "three hit string" if it was 3 hit string it thinks its talking about the 3 MOVE
doc = nlp1("punch string that is safe on block")
doc1 = nlp1("high damage mid punch heat engager")
doc2 = nlp1("three hit string that starts with a low")
doc6 = nlp1("safe on block mid from stance")

for x in doc6:
    print(x.start)

spacy.displacy.render(doc, style="ent", jupyter=True) 
spacy.displacy.render(doc1, style="ent", jupyter=True) 
spacy.displacy.render(doc2, style="ent", jupyter=True) 
spacy.displacy.render(doc6, style="ent", jupyter=True)

AttributeError: 'spacy.tokens.token.Token' object has no attribute 'start'

In [8]:
# results 
# much better results than a few places (just the specific moves part)
# worse at general stuff



In [9]:
# does identify the stance 
doc3 = nlp1("mid poke into high that transition into a stance")
doc5 = nlp1("stance high homing move")
doc7 = nlp1("jumping ch launch low that goes into stance")
# doesnt identify the stance
doc4 = nlp1("high homing move from stance")

print("identifies the stance")
spacy.displacy.render(doc3, style="ent", jupyter=True)
spacy.displacy.render(doc7, style="ent", jupyter=True)
spacy.displacy.render(doc5, style="ent", jupyter=True)
print("does not identify the stance")
spacy.displacy.render(doc4, style="ent", jupyter=True)

identifies the stance


does not identify the stance


In [45]:
tekkenMovesProperties = { 'parry', 'grab', 'armor', 'armored', 'throw', 'homing', 'tornado', 'tornados', 'evasion', 'evading', 'crouch', 'unblockable', 'untechable', 'launcher'}

tekkenOverlaps = {'high', 'low', 'mid', 'heat', 'power', 'guard', 'knock', 'hits', 'is', 'chain', 'stance', 'wall', 'full', 'while'}
overlapsHighLow = {'crush', 'crushing', 'crushes', 'damage'}

def getTekkenProperties2(doc):
    nlp = spacy.load("en_core_web_sm")
    matcher_attr = "TEXT" 
    new_ents = []
    matcher = PhraseMatcher(nlp.vocab, attr = matcher_attr)
    props = ['high crushes', 'high crushing', 'high crush','low crush', 'low crushing', 'low crushes', 'guard break', 'grounded', 'flip over', 'in crouch',  'chargable', 'holdable', 'blue spark', 'blue sparks', 'knocks down']
    moves = ['heat burst', 'heat smash', 'chain throw', 'chain grab', 'power crush','homing', 'armor', 'armored', 'parry', 'sabaki', 'instant screw', 'instant tornado', 'rage art']
    patterns = [nlp.make_doc(text) for text in props]
    movePatterns = [nlp.make_doc(text) for text in moves]
    matcher.add("PROPERTIES", patterns)
    matcher.add("MOVE_TYPES", movePatterns)
    matches = matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]
        # property
        if match_id == 17494066980514271985:
            span = doc.char_span(span.start_char, span.end_char, label = "MOVE_PROPERTY")
        # move type
        elif match_id == 9842506821441691883:
            span = doc.char_span(span.start_char, span.end_char, label = "MOVE_TYPE")
        if span is not None:
            new_ents.append(span)

    doc.ents = list(doc.ents) + new_ents

testDoc = nlp1("low, high, mid string that high crushes on the first hit, knocks down on hit, heat smash")
#getTekkenProperties(testDoc)
#print(list(newTestDoc.ents))
#getTekkenProperties2(newTestDoc)
#print(list(newTestDoc.ents))
#print(list(testDoc.ents))



[]
[high crushes, heat smash]
